<a href="https://colab.research.google.com/github/abakm/AL-ML_Assignment-1/blob/master/IntermediateCodingAssessment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [103]:
# import statements
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV

In [104]:
train_df = pd.read_csv('./train_LZdllcl.csv')
test_df =pd.read_csv('./test_2umaH9m.csv')
sub_df = pd.read_csv('./sample_submission_M0L0uXE.csv')


print(train_df.head())
print(test_df.head())
print(sub_df.head())

   employee_id         department     region         education gender  \
0        65438  Sales & Marketing   region_7  Master's & above      f   
1        65141         Operations  region_22        Bachelor's      m   
2         7513  Sales & Marketing  region_19        Bachelor's      m   
3         2542  Sales & Marketing  region_23        Bachelor's      m   
4        48945         Technology  region_26        Bachelor's      m   

  recruitment_channel  no_of_trainings  age  previous_year_rating  \
0            sourcing                1   35                   5.0   
1               other                1   30                   5.0   
2            sourcing                1   34                   3.0   
3               other                2   39                   1.0   
4               other                1   45                   3.0   

   length_of_service  KPIs_met >80%  awards_won?  avg_training_score  \
0                  8              1            0                  49   
1 

In [105]:
# exploratory Analysis
print(f"Dataset Shape: {train_df.shape}")
print(f"Number of Rows: {train_df.shape[0]}")
print(f"Number of Columns: {train_df.shape[1]}")
print()

print("Column Names and Data Types:")
print(train_df.dtypes)
print()

print("First 5 rows:")
print(train_df.head())
print()

print("Last 5 rows:")
print(train_df.tail())
print()

print("Dataset Info:")
print(train_df.info())

Dataset Shape: (54808, 14)
Number of Rows: 54808
Number of Columns: 14

Column Names and Data Types:
employee_id               int64
department               object
region                   object
education                object
gender                   object
recruitment_channel      object
no_of_trainings           int64
age                       int64
previous_year_rating    float64
length_of_service         int64
KPIs_met >80%             int64
awards_won?               int64
avg_training_score        int64
is_promoted               int64
dtype: object

First 5 rows:
   employee_id         department     region         education gender  \
0        65438  Sales & Marketing   region_7  Master's & above      f   
1        65141         Operations  region_22        Bachelor's      m   
2         7513  Sales & Marketing  region_19        Bachelor's      m   
3         2542  Sales & Marketing  region_23        Bachelor's      m   
4        48945         Technology  region_26        Bache

In [106]:

# checking for null values
print(train_df.isnull().sum())



train_df['education'] = train_df['education'].fillna(train_df['education'].mode()[0])
train_df['previous_year_rating'] = train_df['previous_year_rating'].fillna(train_df['previous_year_rating'].mode()[0])
train_df.drop_duplicates(inplace=True)


employee_id                0
department                 0
region                     0
education               2409
gender                     0
recruitment_channel        0
no_of_trainings            0
age                        0
previous_year_rating    4124
length_of_service          0
KPIs_met >80%              0
awards_won?                0
avg_training_score         0
is_promoted                0
dtype: int64


In [107]:

# checking for null values
print(test_df.isnull().sum())
test_df['education'] = test_df['education'].fillna(test_df['education'].mode()[0])
test_df['previous_year_rating'] = test_df['previous_year_rating'].fillna(test_df['previous_year_rating'].mode()[0])
test_df.drop_duplicates(inplace=True)


employee_id                0
department                 0
region                     0
education               1034
gender                     0
recruitment_channel        0
no_of_trainings            0
age                        0
previous_year_rating    1812
length_of_service          0
KPIs_met >80%              0
awards_won?                0
avg_training_score         0
dtype: int64


In [108]:
# Label Encoding

le = LabelEncoder()
train_df['gender'] = le.fit_transform(train_df['gender'])
train_df['department'] = le.fit_transform(train_df['department'])
train_df['region'] = le.fit_transform(train_df['region'])
train_df['education'] = le.fit_transform(train_df['education'])
train_df['recruitment_channel'] = le.fit_transform(train_df['recruitment_channel'])


test_df['gender'] = le.fit_transform(test_df['gender'])
test_df['department'] = le.fit_transform(test_df['department'])
test_df['region'] = le.fit_transform(test_df['region'])
test_df['education'] = le.fit_transform(test_df['education'])
test_df['recruitment_channel'] = le.fit_transform(test_df['recruitment_channel'])


train_df.head()

,employee_id,department,region,education,gender,recruitment_channel,no_of_trainings,age,previous_year_rating,length_of_service,KPIs_met >80%,awards_won?,avg_training_score,is_promoted
0,65438,7,31,2,0,2,1,35,5.0,8,1,0,49,0
1,65141,4,14,0,1,0,1,30,5.0,4,0,0,60,0
2,7513,7,10,0,1,2,1,34,3.0,7,0,0,50,0
3,2542,7,15,0,1,0,2,39,1.0,10,0,0,50,0
4,48945,8,18,0,1,0,1,45,3.0,2,0,0,73,0


In [109]:
train_x = train_df.drop(["employee_id", "is_promoted"], axis=1)
train_y = train_df["is_promoted"]

train_x.shape

(54808, 12)

In [110]:
# create model
rc = RandomForestClassifier(criterion='gini', max_depth=50,n_estimators=200)

In [111]:
# fit the model
model = rc.fit(train_x, train_y)

In [112]:
sub_rows = []
for employee_id in sub_df['employee_id']:
  row = test_df[test_df['employee_id'] == employee_id].drop(["employee_id"], axis=1)
  if row.empty:
    row = train_df[train_df['employee_id'] == employee_id].drop(["employee_id", "is_promoted"], axis=1)
  if row.empty:
    continue
  print(model.predict(row))
  sub_rows.append(dict(employee_id=employee_id, is_promoted=model.predict(row)[0]))
predict = pd.DataFrame(sub_rows)
predict.to_csv('submission.csv', index=False)





Streaming output truncated to the last 5000 lines.
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[1]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[1]
[0]
[0]
[0]
[

In [113]:
# # fine tuning
# criterion='gini'
# max_depth=50
# n_estimators=200
# parameters = {
#     'n_estimators': [50, 100, 200],
#     'max_depth': [10, 50, 60],
#     "criterion": ["gini", "entropy"],
# }


# random_search = RandomizedSearchCV(RandomForestClassifier(), parameters, n_iter=5, cv=5, scoring="accuracy", verbose=1, n_jobs=1)
# random_search.fit(train_x, train_y)

# best_params = random_search.best_params_
# best_score = random_search.best_score_

# print("Best Parameters:", best_params)
# print("Best Score:", best_score)

# n_estimators = best_params['n_estimators']
# max_depth = best_params['max_depth']
# criterion = best_params['criterion']